In [2]:
import os
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

## Steps to build pipeline

1. Tokenize the query
2. Tokenize the streams
3. Tokenize the stream

Run LSTM on query

In [112]:
os.listdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/corpus-data/')

['2012_aurora_shooting.tsv.gz',
 '2012_pakistan_garment_factory_fires.tsv.gz',
 'hurricane_sandy.tsv.gz',
 'wisconsin_sikh_temple_shooting.tsv.gz']

In [111]:
os.chdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/corpus-data/')

In [5]:
df1 = pd.read_csv('./2012_aurora_shooting.tsv.gz', sep='\t')
df2 = pd.read_csv('./2012_pakistan_garment_factory_fires.tsv.gz', sep='\t')
df3 = pd.read_csv('./hurricane_sandy.tsv.gz', sep='\t')
df4 = pd.read_csv('./wisconsin_sikh_temple_shooting.tsv.gz', sep='\t')

In [6]:
# Title is the title of the article
print df1[df1['stream id']=='1342767285-e9b3e7dc6a9e31c8b2301848ab082117'].title[1]
# Each text is a stream (i.e., a sentence from the article)
print df1[df1['stream id']=='1342767285-e9b3e7dc6a9e31c8b2301848ab082117'].text[1]

Monument man to ride in Courage Classic for son - OutThereColorado.com
They 're cycling for those they care for , for those they know , and for those they nurture .


In [7]:
df1.head()

,hour,score,stream id,url,title,sentence,text
0,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,0,Jason Mezey -LRB- right -RRB- and fellow Coura...
1,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,1,"They 're cycling for those they care for , for..."
2,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,2,"For one cyclist , Monument resident Jason Meze..."
3,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,3,"The infant has one full left arm , half of his..."
4,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,4,"`` Dillon is a normally functioning , healthy ..."


In [8]:
df2.head()

,hour,score,stream id,url,title,sentence,text
0,0,14.635408,1347369130-aa4ff946adb6599b137feb38281c4018,http://menmedia.co.uk/manchestereveningnews/ne...,Trio arrested over fire which destroyed Radcli...,0,Three men have been arrested on suspicion of a...
1,0,14.635408,1347369130-aa4ff946adb6599b137feb38281c4018,http://menmedia.co.uk/manchestereveningnews/ne...,Trio arrested over fire which destroyed Radcli...,1,"The men - aged 26 , 21 and 18 - have been ques..."
2,0,14.635408,1347369130-aa4ff946adb6599b137feb38281c4018,http://menmedia.co.uk/manchestereveningnews/ne...,Trio arrested over fire which destroyed Radcli...,2,Around 100 homes had to be evacuated because o...
3,0,14.635408,1347369130-aa4ff946adb6599b137feb38281c4018,http://menmedia.co.uk/manchestereveningnews/ne...,Trio arrested over fire which destroyed Radcli...,3,No-one was hurt .
4,0,14.635408,1347369130-aa4ff946adb6599b137feb38281c4018,http://menmedia.co.uk/manchestereveningnews/ne...,Trio arrested over fire which destroyed Radcli...,4,Forensic investigators from Greater Manchester...


In [9]:
df3.head()

,hour,score,stream id,url,title,sentence,text
0,0,27.31798,1351093540-7889486104cb75da1b70dbca8cd22296,http://www2.wspa.com/news/2012/oct/24/2/soaked...,Sandy Reaches Hurricane Status | WSPA,0,hurricane Sandy as of 11:00 a.m. Wednesday .
1,0,27.31798,1351093540-7889486104cb75da1b70dbca8cd22296,http://www2.wspa.com/news/2012/oct/24/2/soaked...,Sandy Reaches Hurricane Status | WSPA,1,Sandy is now a category one hurricane .
2,0,27.31798,1351093540-7889486104cb75da1b70dbca8cd22296,http://www2.wspa.com/news/2012/oct/24/2/soaked...,Sandy Reaches Hurricane Status | WSPA,2,The National Hurricane Center reported Hurrica...
3,0,27.31798,1351093540-7889486104cb75da1b70dbca8cd22296,http://www2.wspa.com/news/2012/oct/24/2/soaked...,Sandy Reaches Hurricane Status | WSPA,3,A hurricane warning is in effect for Jamaica a...
4,0,27.31798,1351093540-7889486104cb75da1b70dbca8cd22296,http://www2.wspa.com/news/2012/oct/24/2/soaked...,Sandy Reaches Hurricane Status | WSPA,4,The government of the Bahamas has issued a hur...


In [10]:
df4.head()

,hour,score,stream id,url,title,sentence,text
0,0,21.3082,1344181816-4d5484bdb6193dc2a2b198803644c27c,http://www.thisisleicestershire.co.uk/Solar-po...,Solar power at Sikh temple | This is Leicester...,0,A Sikh temple is harnessing the sun 's rays to...
1,0,21.3082,1344181816-4d5484bdb6193dc2a2b198803644c27c,http://www.thisisleicestershire.co.uk/Solar-po...,Solar power at Sikh temple | This is Leicester...,1,"Leicester 's Guru Tegh Bahadur Gurdwara , in E..."
2,0,21.3082,1344181816-4d5484bdb6193dc2a2b198803644c27c,http://www.thisisleicestershire.co.uk/Solar-po...,Solar power at Sikh temple | This is Leicester...,2,"The # 125,000 environmental project has been f..."
3,0,21.3082,1344181816-4d5484bdb6193dc2a2b198803644c27c,http://www.thisisleicestershire.co.uk/Solar-po...,Solar power at Sikh temple | This is Leicester...,3,"Surinderpal Singh Rai , general secretary of t..."
4,0,21.3082,1344181816-4d5484bdb6193dc2a2b198803644c27c,http://www.thisisleicestershire.co.uk/Solar-po...,Solar power at Sikh temple | This is Leicester...,4,He said : `` It 's a huge four-storey building...


In [113]:
os.listdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/trec-2013-data/')

['matches.tsv.gz',
 'nuggets.tsv.gz',
 'trec2013-ts-topics-test.xml',
 'updates.tsv.gz']

In [12]:
os.chdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/trec-2013-data/')

In [13]:
df5 = pd.read_csv('./nuggets.tsv.gz', sep='\t')
df6 = pd.read_csv('./matches.tsv.gz', sep='\t')
df7 = pd.read_csv('./updates.tsv.gz', sep='\t')

In [14]:
df5.head() # Nuggets

,query_id,nugget_id,timestamp,importance,nugget_len,nugget_text
0,TS13.1,VMTS13.01.050,1329945286,3,6,"train accident in Buenos Aires, Argentina."
1,TS13.1,VMTS13.01.051,1329945286,3,4,unknown number were killed
2,TS13.1,VMTS13.01.052,1329945286,3,2,Hundreds injured
3,TS13.1,VMTS13.01.053,1329946985,1,2,in 2012
4,TS13.1,VMTS13.01.054,1329947346,1,3,"February 22, 2012"


In [15]:
df6.head() # Matches

query_id    update_id   nugget_id  \
TS13.1 1330003205-a361ee100c9b058a0f0f4355cec64047-0                     VMTS13.01.077   
       1329996913-06251bb5df849f4e5efe4245acb1c342-2                     VMTS13.01.050   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-1                     VMTS13.01.050   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0                     VMTS13.01.078   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0                     VMTS13.01.052   

                                                      match_start  match_end  \
TS13.1 1330003205-a361ee100c9b058a0f0f4355cec64047-0           35         53   
       1329996913-06251bb5df849f4e5efe4245acb1c342-2           50         74   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-1           50        101   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0            0          8   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0            9         25   

                                                      auto_p  
TS13.1 1330003205-a361ee100c9b058a0f0f4355cec64047-0       0  
       1329996913-06251bb5df849f4e5efe4245acb1c342-2       0  
       1329996320-a8bb0847959520d7f32b2b12d486e33e-1       0  
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0       0  
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0       0

In [16]:
df7.head() # Updates

,query_id,update_id,doc_id,sentence_id,update_len,duplicate_id,update_text
0,TS13.1,1329910380-3afda7882974e306bc75176f5ce37f3e-5,1329910380-3afda7882974e306bc75176f5ce37f3e,5,922,NaN,"The soldiers, Mr Massimiliano Latorre and Mr S..."
1,TS13.1,1329910549-b5765884364f2991653f58c01a85b6c7-1,1329910549-b5765884364f2991653f58c01a85b6c7,1,190,NaN,"“At the same time, the UM system Actors David ..."
2,TS13.1,1329911230-5599b4c78673045d4522f23f2a357509-7,1329911230-5599b4c78673045d4522f23f2a357509,7,24,NaN,They are protesting the English by vandalising...
3,TS13.1,1329911327-9bcecf79f6aee904fd6e962cad884b6b-23,1329911327-9bcecf79f6aee904fd6e962cad884b6b,23,14,NaN,But it was one of the biggest box office smash...
4,TS13.1,1329911633-579d7af8f8ae26a964b05b69ae375a85-24,1329911633-579d7af8f8ae26a964b05b69ae375a85,24,65,NaN,Lyle’ s article is worth reading in its entire...


In [17]:
df7['query_id'].nunique()

9

In [18]:
f = open('./trec2013-ts-topics-test.xml', 'rb')

out = f.readlines()
ox = BeautifulSoup(''.join(out),'lxml').contents[1]

In [41]:
i.findAll('title')[0].text, ',',i.findAll('query')[0].text

(u'2012 Buenos Aires Rail Disaster', ',', u'buenos aires train crash')

In [42]:
print "Wiki article, Query"
qs = []
c = 0
for i in ox.findAll('event'):
    qs.append(i.findAll('query')[0].text)
    print c, i.findAll('title')[0].text, ',',i.findAll('query')[0].text
    c+=1


Wiki article, Query
0 2012 Buenos Aires Rail Disaster , buenos aires train crash
1 2012 Pakistan garment factory fires , pakistan factory fire
2 2012 Aurora shooting , colorado shooting
3 Wisconsin Sikh temple shooting , sikh temple shooting
4 Hurricane Isaac (2012) , hurricane isaac
5 Hurricane Sandy , hurricane sandy
6 June 2012 North American derecho , midwest derecho
7 Typhoon Bopha , typhoon bopha
8 2012 Guatemala earthquake , guatemala earthquake
9 2012 Tel Aviv bus bombing , tel aviv bus bombing


# Only have ten events, so we should build the code to be more general

## This is the stream data for Aurora -- all the same query

In [24]:
# This is the aurora shooting, query #2 in the xml
df1.head()

,hour,score,stream id,url,title,sentence,text
0,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,0,Jason Mezey -LRB- right -RRB- and fellow Coura...
1,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,1,"They 're cycling for those they care for , for..."
2,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,2,"For one cyclist , Monument resident Jason Meze..."
3,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,3,"The infant has one full left arm , half of his..."
4,0,13.591976,1342767285-e9b3e7dc6a9e31c8b2301848ab082117,http://articles.outtherecolorado.com/articles/...,Monument man to ride in Courage Classic for so...,4,"`` Dillon is a normally functioning , healthy ..."


## these are the nuggets for all of the trek data

In [30]:
df5.shape

(1340, 6)

In [31]:
df5.head()

,query_id,nugget_id,timestamp,importance,nugget_len,nugget_text
0,TS13.1,VMTS13.01.050,1329945286,3,6,"train accident in Buenos Aires, Argentina."
1,TS13.1,VMTS13.01.051,1329945286,3,4,unknown number were killed
2,TS13.1,VMTS13.01.052,1329945286,3,2,Hundreds injured
3,TS13.1,VMTS13.01.053,1329946985,1,2,in 2012
4,TS13.1,VMTS13.01.054,1329947346,1,3,"February 22, 2012"


In [38]:
df5[df5['nugget_id']=='VMTS13.01.077'] # Nuggets

,query_id,nugget_id,timestamp,importance,nugget_len,nugget_text
26,TS13.1,VMTS13.01.077,1329977992,1,2,676+ injuries


Index in df6 (matches) is concatenation of (queryid, streamid, and sentence)

In [33]:
df6.shape

(5157, 4)

In [32]:
df6.head()

query_id    update_id   nugget_id  \
TS13.1 1330003205-a361ee100c9b058a0f0f4355cec64047-0                     VMTS13.01.077   
       1329996913-06251bb5df849f4e5efe4245acb1c342-2                     VMTS13.01.050   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-1                     VMTS13.01.050   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0                     VMTS13.01.078   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0                     VMTS13.01.052   

                                                      match_start  match_end  \
TS13.1 1330003205-a361ee100c9b058a0f0f4355cec64047-0           35         53   
       1329996913-06251bb5df849f4e5efe4245acb1c342-2           50         74   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-1           50        101   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0            0          8   
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0            9         25   

                                                      auto_p  
TS13.1 1330003205-a361ee100c9b058a0f0f4355cec64047-0       0  
       1329996913-06251bb5df849f4e5efe4245acb1c342-2       0  
       1329996320-a8bb0847959520d7f32b2b12d486e33e-1       0  
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0       0  
       1329996320-a8bb0847959520d7f32b2b12d486e33e-0       0

## These are the matches of the nuggets

In [36]:
df7.head()

,query_id,update_id,doc_id,sentence_id,update_len,duplicate_id,update_text
0,TS13.1,1329910380-3afda7882974e306bc75176f5ce37f3e-5,1329910380-3afda7882974e306bc75176f5ce37f3e,5,922,NaN,"The soldiers, Mr Massimiliano Latorre and Mr S..."
1,TS13.1,1329910549-b5765884364f2991653f58c01a85b6c7-1,1329910549-b5765884364f2991653f58c01a85b6c7,1,190,NaN,"“At the same time, the UM system Actors David ..."
2,TS13.1,1329911230-5599b4c78673045d4522f23f2a357509-7,1329911230-5599b4c78673045d4522f23f2a357509,7,24,NaN,They are protesting the English by vandalising...
3,TS13.1,1329911327-9bcecf79f6aee904fd6e962cad884b6b-23,1329911327-9bcecf79f6aee904fd6e962cad884b6b,23,14,NaN,But it was one of the biggest box office smash...
4,TS13.1,1329911633-579d7af8f8ae26a964b05b69ae375a85-24,1329911633-579d7af8f8ae26a964b05b69ae375a85,24,65,NaN,Lyle’ s article is worth reading in its entire...


In [67]:
texts = [ q.split(" ") for q in qs ]

In [72]:
from gensim import corpora
from collections import defaultdict

frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [ [token for token in text ]  for text in texts]
dictionary = corpora.Dictionary(texts)

In [91]:
# Reversing the dictionary
word2idx = dict((k, v) for v, k in dictionary.items())

In [100]:
len([0, 51, 27, 16, 10, 856, 53, 25, 34, 69]), len([51, 27, 16, 10, 856, 53, 25, 34, 69, 1])

(10, 10)

In [73]:
dictionary.token2id

{u'aires': 2,
 u'aviv': 20,
 u'bombing': 23,
 u'bopha': 17,
 u'buenos': 3,
 u'bus': 21,
 u'colorado': 8,
 u'crash': 1,
 u'derecho': 14,
 u'earthquake': 19,
 u'factory': 6,
 u'fire': 4,
 u'guatemala': 18,
 u'hurricane': 12,
 u'isaac': 11,
 u'midwest': 15,
 u'pakistan': 5,
 u'sandy': 13,
 u'shooting': 7,
 u'sikh': 9,
 u'tel': 22,
 u'temple': 10,
 u'train': 0,
 u'typhoon': 16}

In [75]:
qs

[u'buenos aires train crash',
 u'pakistan factory fire',
 u'colorado shooting',
 u'sikh temple shooting',
 u'hurricane isaac',
 u'hurricane sandy',
 u'midwest derecho',
 u'typhoon bopha',
 u'guatemala earthquake',
 u'tel aviv bus bombing']

In [76]:
texts

[[u'buenos', u'aires', u'train', u'crash'],
 [u'pakistan', u'factory', u'fire'],
 [u'colorado', u'shooting'],
 [u'sikh', u'temple', u'shooting'],
 [u'hurricane', u'isaac'],
 [u'hurricane', u'sandy'],
 [u'midwest', u'derecho'],
 [u'typhoon', u'bopha'],
 [u'guatemala', u'earthquake'],
 [u'tel', u'aviv', u'bus', u'bombing']]

In [96]:
text_numindex = [ [word2idx[i] for i in t] for t in texts]

In [97]:
texts[0], text_numindex[0]

([u'buenos', u'aires', u'train', u'crash'], [3, 2, 0, 1])

In [98]:
dictionary[3], dictionary[2], dictionary[0], dictionary[1]

(u'buenos', u'aires', u'train', u'crash')

In [103]:
# This is the thing we have to export
text_numindex

[[3, 2, 0, 1],
 [5, 6, 4],
 [8, 7],
 [9, 10, 7],
 [12, 11],
 [12, 13],
 [15, 14],
 [16, 17],
 [18, 19],
 [22, 20, 21, 23]]

# This section tokenizes each dataset into a list of numeric indices

In [178]:
from gensim import corpora
from collections import defaultdict
import sys
import pickle
import csv
reload(sys)

def BuildIndexFile(infilename, outfilename):
    """
    :type  infilename: str
    :param infilename: Fulll name of input file 

    :type  outfilename: str
    :param outfilename: Export filename, without the '.csv'
    """
    sys.setdefaultencoding('utf-8')

    print('Loading %s' % infilename)
    
    df = pd.read_csv(infilename, sep='\t', encoding='latin-1')
    texts = [ t.split(" ") for t in df['text'] ]

    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    texts = [ [token for token in text ]  for text in texts]
    dictionary = corpora.Dictionary(texts)

    # Reversing the dictionary
    word2idx = dict((k, v) for v, k in dictionary.items())    
    # Mapping to numeric list
    text_numindex = [ [word2idx[i] for i in t] for t in texts]

    
    # Exporting the dictionaries
    print("Exporting word to index and dictionary to word indices")
    output = open(outfilename+'_w2idx.pkl', 'ab+')
    pickle.dump(word2idx, output)
    output.close()

    output = open(outfilename+'_idx2w.pkl', 'ab+')
    pickle.dump(dictionary.id2token, output)
    output.close()

    # Exporting files
    print('...file export to %s' % outfilename)
    with open(outfilename+'_numtext.csv', 'wb') as csvfile:
        data = csv.writer(csvfile, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        data.writerow(['Text'])
        data.writerows(text_numindex)

    csvfile.close()

In [186]:
os.chdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/corpus-data/')
os.listdir('/Users/franciscojavierarceo/GitHub/DeepNLPQLearning/DO_NOT_UPLOAD_THIS_DATA/corpus-data/')

['0-output',
 '2012_aurora_shooting.tsv.gz',
 '2012_pakistan_garment_factory_fires.tsv.gz',
 'hurricane_sandy.tsv.gz',
 'wisconsin_sikh_temple_shooting.tsv.gz']

In [179]:
BuildIndexFile('./2012_aurora_shooting.tsv.gz', './0-output/2012_aurora_shooting')

In [187]:
BuildIndexFile('./2012_pakistan_garment_factory_fires.tsv.gz', './0-output/2012_pakistan_garment_factory_fires')

In [188]:
BuildIndexFile('./hurricane_sandy.tsv.gz', './0-output/hurricane_sandy')

In [189]:
BuildIndexFile('./wisconsin_sikh_temple_shooting.tsv.gz', './0-output/wisconsin_sikh_temple_shooting')

In [191]:
as_widx = pickle.load(open('./0-output/2012_aurora_shooting_w2idx.pkl', 'rb'))
as_idxw = pickle.load(open('./0-output/2012_aurora_shooting_idx2w.pkl', 'rb'))

In [192]:
pg_widx = pickle.load(open('./0-output/2012_pakistan_garment_factory_fires_w2idx.pkl', 'rb'))
pg_idxw = pickle.load(open('./0-output/2012_pakistan_garment_factory_fires_idx2w.pkl', 'rb'))

In [193]:
hs_widx = pickle.load(open('./0-output/hurricane_sandy_w2idx.pkl', 'rb'))
hs_idxw = pickle.load(open('./0-output/hurricane_sandy_idx2w.pkl', 'rb'))

In [194]:
ws_widx = pickle.load(open('./0-output/wisconsin_sikh_temple_shooting_w2idx.pkl', 'rb'))
ws_idxw = pickle.load(open('./0-output/wisconsin_sikh_temple_shooting_idx2w.pkl', 'rb'))